In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data_sources = {}
data_preproc = {}
id_dict = {}

In [3]:
hippie = pd.read_csv('../data/network/hippie_current.txt', sep='\t', header=None)

data_sources['hippie'] = hippie
data_preproc['hippie'] = {
    'columns':[0,1],
    'maps':[
        lambda x: id_dict[x.split('_')[0]],
        lambda x: id_dict[x.split('_')[0]]
    ]
}

print(hippie.columns)
hippie.head()

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')


,0,1,2,3,4,5
0,AL1A1_HUMAN,216,AL1A1_HUMAN,216,0.76,"experiments:in vivo,Two-hybrid;pmids:12081471,..."
1,ITA7_HUMAN,3679,ACHA_HUMAN,1134,0.73,"experiments:in vivo,Affinity Capture-Western,a..."
2,NEB1_HUMAN,55607,ACTG_HUMAN,71,0.65,"experiments:in vitro,in vivo;pmids:9362513,120..."
3,SRGN_HUMAN,5552,CD44_HUMAN,960,0.63,"experiments:in vivo;pmids:9334256,16189514,167..."
4,GRB7_HUMAN,2886,ERBB2_HUMAN,2064,0.90,"experiments:in vitro,in vivo,Reconstituted Com..."


In [4]:
phosphosite_substrate = pd.read_csv('../data/network/Kinase_Substrate_Dataset.txt', sep='\t', skiprows=2)

data_sources['phosphosite_substrate'] = phosphosite_substrate
data_preproc['phosphosite_substrate'] = {
    'columns':['GENE','SUBSTRATE'],
    'maps':[
        lambda x: id_dict[x.upper()],
        lambda x: id_dict[x.upper()]
    ]
}

print(phosphosite_substrate.columns)
display(phosphosite_substrate.head())


phosphosite_regulatory = pd.read_csv('../data/network/Regulatory_sites.txt', sep='\t', skiprows=2, error_bad_lines=False)

data_sources['phosphosite_regulatory'] =  phosphosite_regulatory
data_preproc['phosphosite_regulatory'] = {
    'columns':['GENE','SUBSTRATE'],
    'maps':[
        lambda x: id_dict[x.upper()],
        lambda x: id_dict[x.upper()]
    ]
}

print(phosphosite_regulatory.columns)
display(phosphosite_substrate.head())

Index(['GENE', 'KINASE', 'KIN_ACC_ID', 'KIN_ORGANISM', 'SUBSTRATE',
       'SUB_GENE_ID', 'SUB_ACC_ID', 'SUB_GENE', 'SUB_ORGANISM', 'SUB_MOD_RSD',
       'SITE_GRP_ID', 'SITE_+/-7_AA', 'DOMAIN', 'IN_VIVO_RXN', 'IN_VITRO_RXN',
       'CST_CAT#'],
      dtype='object')


,GENE,KINASE,KIN_ACC_ID,KIN_ORGANISM,SUBSTRATE,SUB_GENE_ID,SUB_ACC_ID,SUB_GENE,SUB_ORGANISM,SUB_MOD_RSD,SITE_GRP_ID,SITE_+/-7_AA,DOMAIN,IN_VIVO_RXN,IN_VITRO_RXN,CST_CAT#
0,Pak2,PAK2,Q64303,rat,MEK1,170851.0,Q01986,Map2k1,rat,S298,448284,RtPGRPLsSYGMDSR,Pkinase,,X,9128; 98195
1,Pak2,PAK2,Q64303,rat,PRKD1,85421.0,Q9WTQ1,Prkd1,rat,S203,449896,GVRRRRLsNVsLTGL,NaN,X,,NaN
2,Pak2,PAK2,Q64303,rat,prolactin,5617.0,P01236,PRL,human,S207,451732,LHCLRRDsHKIDNYL,Hormone_1,,X,NaN
3,Pak2,PAK2,Q64303,rat,prolactin,24683.0,P01237,Prl,rat,S206,451732,IRCLRRDsHKVDNYL,Hormone_1,,X,NaN
4,EIF2AK1,HRI,Q9BQI3,human,eIF2-alpha,54318.0,P68101,Eif2s1,rat,S52,447635,MILLSELsRRRIRSI,S1,,X,3597; 9721; 3398; 5199


Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19'],
      dtype='object')


,GENE,KINASE,KIN_ACC_ID,KIN_ORGANISM,SUBSTRATE,SUB_GENE_ID,SUB_ACC_ID,SUB_GENE,SUB_ORGANISM,SUB_MOD_RSD,SITE_GRP_ID,SITE_+/-7_AA,DOMAIN,IN_VIVO_RXN,IN_VITRO_RXN,CST_CAT#
0,Pak2,PAK2,Q64303,rat,MEK1,170851.0,Q01986,Map2k1,rat,S298,448284,RtPGRPLsSYGMDSR,Pkinase,,X,9128; 98195
1,Pak2,PAK2,Q64303,rat,PRKD1,85421.0,Q9WTQ1,Prkd1,rat,S203,449896,GVRRRRLsNVsLTGL,NaN,X,,NaN
2,Pak2,PAK2,Q64303,rat,prolactin,5617.0,P01236,PRL,human,S207,451732,LHCLRRDsHKIDNYL,Hormone_1,,X,NaN
3,Pak2,PAK2,Q64303,rat,prolactin,24683.0,P01237,Prl,rat,S206,451732,IRCLRRDsHKVDNYL,Hormone_1,,X,NaN
4,EIF2AK1,HRI,Q9BQI3,human,eIF2-alpha,54318.0,P68101,Eif2s1,rat,S52,447635,MILLSELsRRRIRSI,S1,,X,3597; 9721; 3398; 5199


In [5]:
hprd = pd.read_csv('../data/network/HPRD_ALL_BINARY_PROTEIN_PROTEIN_INTERACTIONS.txt', sep='\t', header=None)

data_sources['hprd'] = hprd
data_preproc['hprd'] = {
    'columns':['GENE','SUBSTRATE'],
    'maps':[
        lambda x: x.upper(),
        lambda x: x.upper()
    ]
}

print(hprd.columns)
hprd.head()

Int64Index([0, 1, 2, 3, 4, 5, 6, 7], dtype='int64')


,0,1,2,3,4,5,6,7
0,ALDH1A1,1,NP_000680.2,ALDH1A1,1,NP_000680.2,in vivo;yeast 2-hybrid,"1,208,147,116,189,510"
1,ITGA7,2761,NP_001138468.1,CHRNA1,7,NP_001034612.1,in vivo,10910772
2,PPP1R9A,16000,NP_060120.2,ACTG1,17,NP_001605.1,in vitro;in vivo,"936,251,312,052,877"
3,SRGN,1513,NP_002718.2,CD44,115,NP_000601.3,in vivo,9334256
4,GRB7,3311,NP_005301.2,ERBB2,1281,NP_004439.2,in vitro;in vivo,9079677


In [6]:
reactome = pd.read_csv('../data/network/reactome.homo_sapiens.interactions.tab-delimited.txt', sep='\t')

data_sources['reactome'] = reactome
data_preproc['reactome'] = {
    'columns': ['# Interactor 1 uniprot id','Interactor 2 uniprot id'],
    'maps': [
        lambda x: id_dict[x.split(':')[1]],
        lambda x: id_dict[x.split(':')[1]]
    ]
}

print(reactome.columns)
reactome.head()

Index(['# Interactor 1 uniprot id', 'Interactor 1 Ensembl gene id',
       'Interactor 1 Entrez Gene id', 'Interactor 2 uniprot id',
       'Interactor 2 Ensembl gene id', 'Interactor 2 Entrez Gene id',
       'Interaction type', 'Interaction context', 'Pubmed references'],
      dtype='object')


,# Interactor 1 uniprot id,Interactor 1 Ensembl gene id,Interactor 1 Entrez Gene id,Interactor 2 uniprot id,Interactor 2 Ensembl gene id,Interactor 2 Entrez Gene id,Interaction type,Interaction context,Pubmed references
0,uniprotkb:Q9Y287,ENSEMBL:ENSG00000136156,entrezgene/locuslink:9445,uniprotkb:Q9Y287,ENSEMBL:ENSG00000136156,entrezgene/locuslink:9445,physical association,reactome:R-HSA-976871,14690516|10391242
1,uniprotkb:P37840,ENSEMBL:ENSG00000145335,entrezgene/locuslink:6622,uniprotkb:P37840,ENSEMBL:ENSG00000145335,entrezgene/locuslink:6622,physical association,reactome:R-HSA-1247852,24243840
2,uniprotkb:P0DJI8,ENSEMBL:ENSG00000173432,entrezgene/locuslink:6288,uniprotkb:P0DJI8,ENSEMBL:ENSG00000173432,entrezgene/locuslink:6288,physical association,reactome:R-HSA-976898,19393650|103558
3,uniprotkb:P06727,ENSEMBL:ENSG00000110244,entrezgene/locuslink:337,uniprotkb:P06727,ENSEMBL:ENSG00000110244,entrezgene/locuslink:337,physical association,reactome:R-HSA-976889,15146166
4,uniprotkb:P01160,ENSEMBL:ENSG00000175206,entrezgene/locuslink:4878,uniprotkb:P01160,ENSEMBL:ENSG00000175206,entrezgene/locuslink:4878,physical association,reactome:R-HSA-976987,2142465|2945573


In [7]:
'''biogrid = pd.read_csv('../data/newtork/', sep='\t')

data_sources['biogrid'] = biogrid
data_preproc['biogrid'] = {
    'columns': ['',''],
    'maps': [
        lambda x: x,
        lambda x: x
    ]
}

print(biogrid.columns)
biogrid.head()'''

"biogrid = pd.read_csv('../data/newtork/', sep='\t')\n\ndata_sources['biogrid'] = biogrid\ndata_preproc['biogrid'] = {\n    'columns': ['',''],\n    'maps': [\n        lambda x: x,\n        lambda x: x\n    ]\n}\n\nprint(biogrid.columns)\nbiogrid.head()"

In [8]:
i2d = pd.read_csv('../data/network/i2d.2_9.txt', sep='\t')

data_sources['i2d'] = i2d
data_preproc['i2d'] = {
    'columns': ['SwissProt1','SwissProt2'],
    'maps': [
        lambda x: id_dict[x],
        lambda x: id_dict[x]
    ]
}

print(i2d.columns)
i2d.head()

Index(['Dataset', 'SwissProt1', 'SwissProt2'], dtype='object')


,Dataset,SwissProt1,SwissProt2
0,SOURAV_MAPK_LOW,P63000,A0AUZ9
1,IntAct,Q96CV9,A0AUZ9
2,BioGrid,P0CG48,A0AV96
3,IntAct_Mouse,P62258,A0AV96
4,IntAct_Mouse,P63104,A0AV96


In [9]:
mint = pd.read_csv('../data/network/species_human.txt', sep='\t', header=None)

data_sources['mint'] = mint
data_preproc['mint'] = {
    'columns': [0,1],
    'maps':[
        lambda x: id_dict[x.split(':')[1]],
        lambda x: id_dict[x.split(':')[1]]
    ]
}

print(mint.columns)
mint.head()

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,uniprotkb:Q72547,uniprotkb:Q72547,intact:EBI-7484755|intact:MINT-8208544,intact:EBI-7484755|intact:MINT-8208544,psi-mi:q72547_9hiv1(display_long)|uniprotkb:po...,psi-mi:q72547_9hiv1(display_long)|uniprotkb:po...,"psi-mi:""MI:0030""(cross-linking study)",Nishitsuji et al. (2011),pubmed:22004763|imex:IM-16791|mint:MINT-820698...,taxid:11676(9hiv1)|taxid:11676(Human immunodef...,taxid:11676(9hiv1)|taxid:11676(Human immunodef...,"psi-mi:""MI:0915""(physical association)","psi-mi:""MI:0471""(MINT)",intact:EBI-7484777|mint:MINT-8206988|imex:IM-1...,intact-miscore:0.40
1,uniprotkb:Q72547,uniprotkb:Q72547,intact:EBI-7484755|intact:MINT-8208544,intact:EBI-7484755|intact:MINT-8208544,psi-mi:q72547_9hiv1(display_long)|uniprotkb:po...,psi-mi:q72547_9hiv1(display_long)|uniprotkb:po...,"psi-mi:""MI:0030""(cross-linking study)",Nishitsuji et al. (2011),pubmed:22004763|imex:IM-16791|mint:MINT-820698...,taxid:11676(9hiv1)|taxid:11676(Human immunodef...,taxid:11676(9hiv1)|taxid:11676(Human immunodef...,"psi-mi:""MI:0915""(physical association)","psi-mi:""MI:0471""(MINT)",intact:EBI-7484916|mint:MINT-8207003|imex:IM-1...,intact-miscore:0.40
2,uniprotkb:Q72547,uniprotkb:Q72547,intact:EBI-7484755|intact:MINT-8208544,intact:EBI-7484755|intact:MINT-8208544,psi-mi:q72547_9hiv1(display_long)|uniprotkb:po...,psi-mi:q72547_9hiv1(display_long)|uniprotkb:po...,"psi-mi:""MI:0030""(cross-linking study)",Nishitsuji et al. (2011),pubmed:22004763|imex:IM-16791|mint:MINT-820698...,taxid:11676(9hiv1)|taxid:11676(Human immunodef...,taxid:11676(9hiv1)|taxid:11676(Human immunodef...,"psi-mi:""MI:0915""(physical association)","psi-mi:""MI:0471""(MINT)",intact:EBI-7484965|mint:MINT-8207016|imex:IM-1...,intact-miscore:0.40
3,uniprotkb:Q90VU7,uniprotkb:Q900A7,intact:EBI-7460704|intact:MINT-5281653,intact:EBI-7460739|intact:MINT-8208185,psi-mi:q90vu7_9hiv1(display_long)|uniprotkb:ne...,psi-mi:q900a7_9hiv1(display_long)|uniprotkb:ta...,"psi-mi:""MI:0416""(fluorescence microscopy)",Sugiyama et al. (2011),pubmed:21970979|imex:IM-16787|mint:MINT-820620...,taxid:11676(9hiv1)|taxid:11676(Human immunodef...,taxid:11676(9hiv1)|taxid:11676(Human immunodef...,"psi-mi:""MI:0403""(colocalization)","psi-mi:""MI:0471""(MINT)",intact:EBI-7460732|mint:MINT-8206271|imex:IM-1...,intact-miscore:0.27
4,uniprotkb:Q2Q067,uniprotkb:Q13015,intact:EBI-9675545,intact:EBI-6269719,psi-mi:q2q067_9dela(display_long)|uniprotkb:HB...,psi-mi:af1q_human(display_long)|uniprotkb:MLLT...,"psi-mi:""MI:0397""(two hybrid array)",Simonis et al. (2012),imex:IM-22977|pubmed:22458338,taxid:11908(humt-)|taxid:11908(Human T-lymphot...,taxid:9606(human)|taxid:9606(Homo sapiens),"psi-mi:""MI:0915""(physical association)","psi-mi:""MI:0471""(MINT)",intact:EBI-9675551|imex:IM-22977-4,intact-miscore:0.49


## Network Assembly

In [10]:
network = pd.DataFrame(columns=['Node1', 'Node2'])
network.head()

,Node1,Node2


In [17]:
for source_name, data in data_sources.items():
    print('Adding', source_name, 'to network')
    
    left_col, right_col = data_preproc[source_name]['columns']
    left_map, right_map = data_preproc[source_name]['maps']
    
    left_nodes = data[left_col].apply(left_map)
    right_nodes = data[right_col].apply(right_map)
    
    network.append(pd.DataFrame(left_nodes, right_nodes), columns=network.columns)

Adding hippie to network


KeyError: 'AL1A1'